# Chapter 48: Schelling’s Segregation Model


In 1969, Thomas C. Schelling developed a simple but striking model of [racial segregation](https://www.rand.org/content/dam/rand/pubs/research_memoranda/2008/RM6014.pdf). In particular, it shows that relatively mild preference for neighbors of similar race can lead in aggregate to the collapse of mixed neighborhoods, and high levels of segregation.

In this lecture, we (in fact you) will build and run a version of Schelling’s model.

### 3 The Model

#### Set-up

* Suppose we have two types of people: orange people and green people. we will assume there are 250 of each type.

* These agents all live on a single unit square. The location of an agent is just a point $(𝑥,𝑦)$, where $0 < 𝑥,𝑦 < 1$.

#### Preferences
* We will say that an agent is happy if half or more of her 10 nearest neighbors are of the same type.

* An important point here is that agents are not averse to living in mixed areas.

#### Behaviour
* Initially, agents are mixed together (integrated). In particular, the initial location of each agent is an independent draw from a bivariate uniform distribution on $𝑆 = (0, 1)^2$.

* We assume that each agent will stay put if they are happy and move if unhappy. The algorithm for moving is as follows:
    1. Draw a random location in $𝑆$;
    2. If happy at new location, move there; 
    3. Else, go to step 1.

### 4 Results

In [2]:
from random import uniform, seed 
from math import sqrt
import matplotlib.pyplot as plt 
%matplotlib inline

seed(42)       # for reproducible random numbers

In [4]:
class Agent:
    def __init__(self, type): 
        self.type = type
        self.draw_location() 
    
    def draw_location(self):
        self.location = uniform(0, 1), uniform(0, 1)
    
    def get_distance(self, other):
        "Computes the euclidean distance between self and other agent." 
        a = (self.location[0] - other.location[0])**2
        b = (self.location[1] - other.location[1])**2
        return sqrt(a + b)

    def happy(self, agents):
        "True if sufficient number of nearest neighbors are of the same type." 
        distances = []
        # distances is a list of pairs (d, agent), where d is distance from agent to self
        for agent in agents:
            if self != agent:
                distance = self.get_distance(agent) 
                distances.append((distance, agent))
        
        # == Sort from smallest to largest, according to distance == #
        distances.sort()
        # == Extract the neighboring agents == #
        neighbors = [agent for d, agent in distances[:num_neighbors]]
        # == Count how many neighbors have the same type as self == # 
        num_same_type = sum(self.type == agent.type for agent in neighbors) 
        
        return num_same_type >= require_same_type
    
    def update(self, agents):
        "If not happy, then randomly choose new locations until happy." 
        while not self.happy(agents):
            self.draw_location()


In [5]:
def plot_distribution(agents, cycle_num):
    "Plot the distribution of agents after cycle_num rounds of the loop." 
    x_values_0, y_values_0 = [], []
    x_values_1, y_values_1 = [], []

    # == Obtain locations of each type == #
    for agent in agents:
        x, y = agent.location 
        if agent.type == 0:
            x_values_0.append(x)
            y_values_0.append(y) 
        else:
            x_values_1.append(x)
            y_values_1.append(y)
    fig, ax = plt.subplots(figsize=(8, 8))
    plot_args = {'markersize': 8, 'alpha': 0.6}
    ax.set_facecolor('azure')
    ax.plot(x_values_0, y_values_0, 'o', markerfacecolor='orange',  **plot_args)
    ax.plot(x_values_1, y_values_1, 'o', markerfacecolor='green', **plot_args)
    ax.set_title(f'Cycle {cycle_num-1}')
    plt.show()


In [ ]:
# == Main == #
num_of_type_0 = 250
num_of_type_1 = 250
num_neighbors = 10   # Number of agents regarded as neighbors
require_same_type = 5    # Want at least this many neighbors to be same type

# == Create a list of agents == #
agents = [Agent(0) for i in range(num_of_type_0)] 
agents.extend(Agent(1) for i in range(num_of_type_1))

count = 1
# == Loop until none wishes to move == # 
while True:
    print('Entering loop ', count) 
    plot_distribution(agents, count) 
    count += 1
    no_one_moved = True
    for agent in agents:
        old_location = agent.location 
        agent.update(agents)
        if agent.location != old_location:
            no_one_moved = False 
    if no_one_moved:
                break
        print('Converged, terminating.')